### COMSM0140: Internet Economics and Financial Technology 2023. Main coursework. 
### Radmehr Ghassabtabarshiadeh

Begin your report here...

A/B testing to determine whether SHVR or ZIP traders perform better using BSE market sessions simulation. Test the null hypothesis that there is no signifacnt difference between the two traders and the alternative hypothesis of there being a signifacnt difference between the two traders with the significance value set to 0.05. A market session will run with the specification mentioned on the brief. start time and end time are set to 0 and 600 respectively. After the simulation are complete, two data frames are created, one with everything the "dump_avgbals" creates and another only taking in the average profit of each trader at a time step. To test the hypothesis, a "T test" was used, this test allows to see whether the observed differences are statistically significant. Some helper functions are below to help do what was described. The functions "make_df" and "ttest" are helper functions for "run_market_sim" which create a dataframe and conduct t-test for each market session. the p value is then added to an array and the average of it returned.

In [124]:
# Some code...
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import random
from scipy import stats
import pandas as pd
import seaborn as sns 

from BSE import market_session

def make_df(path: str):
    df = pd.read_csv(path)
    df.columns =  ['name', 'time', 'curr best bid', 'curr best offer', 'trader1', 'total profit1', 'no. 1', 'avg profit1', 'trader2', 'total profit2', 'no. 2', 'avg profit2', 'err']
    df_profit = df[['avg profit1', 'avg profit2']]
    df_profit.columns = ['SHVR', 'ZIP']
    return df, df_profit

def ttest(df):
    zip_avg = df['ZIP']
    shvr_avg = df['SHVR']
        
    t_statistic, p_value = stats.ttest_ind(zip_avg, shvr_avg)
    return t_statistic, p_value

def run_market_sim(trial_id, no_sessions, R, n, supply_range, demand_range, start_time, end_time, path):

    SHVR_num = (n*R)//100
    zic_num = ((100-R)*n)//100
    buyer_spec = [('SHVR', SHVR_num), ('ZIC', zic_num)]
    seller_spec = [('SHVR', SHVR_num), ('ZIC', zic_num)]
    trader_specs = {'sellers': seller_spec, 'buyers': buyer_spec}
    p_val = []

    trial_id = trial_id
    for _ in range(no_sessions):
        supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [supply_range], 'stepmode': 'fixed'}]
        demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [demand_range], 'stepmode': 'fixed'}]
        order_interval = 60
        order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
                    'interval': order_interval, 'timemode': 'periodic'}
        dump_flags = {'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False,
                    'dump_avgbals': True, 'dump_tape': False}

        verbose = False
        market_session(trial_id, start_time, end_time, trader_specs, order_sched, dump_flags, verbose)
        _, df_profit = make_df(path)
        _, p_value = ttest(df_profit)
        p_val.append(p_value)
    
    return np.mean(p_val)

In [126]:
n50P_avg = run_market_sim(trial_id="n500", no_sessions=500, R=50, n=20, supply_range=(310, 310), demand_range=(250, 490), start_time=0, end_time=60*10, path='n500_avg_balance.csv')
n500P_avg = run_market_sim(trial_id="n50", no_sessions=50, R=50, n=20, supply_range=(310, 310), demand_range=(250, 490), start_time=0, end_time=60*10, path='n50_avg_balance.csv')
print(n50P_avg, n500P_avg)

In [ ]:
_, df50 = make_df('n50_avg_balance.csv')
_, df500 = make_df('n500_avg_balance.csv')
plot_500 = sns.kdeplot(data=df500, fill=True)
plt.show()
plot_50 = sns.kdeplot(data=df50, fill=True)

Seaborn library is used to visualise the data frame to determine wheteher the average profit is normaly distributed or not. As the output show, the data is somewhat normally distributed and can be passed for T-test. Data also demonstrates that ZIP traders perform better when there are less market simulation has happened as they have higher profit but as the number of sessions increase it is observed that SHVR trader perform slightly better.

The average P-value for both market sessions are above the threshold of 0.05 suggesting that there is not enough evidence to reject the null hypothesis. After looking at the data it is observed that the Zip traders does give higher average profit per trade. The results suggest that the profablity of the two are statistically indistinguishable, however from the visualisation it is clear that shvr traders have higher profit.

END OF REPORT. ONLY WORD COUNT BELOW THIS POINT. 

In [ ]:
# Do not edit this code. It will print the word count of your notebook.
import io
from nbformat import current

def printWordCount(filepath):

    with io.open(filepath, 'r', encoding='utf-8') as f:
        nb = current.read(f, 'json')

    word_count = 0
    for cell in nb.worksheets[0].cells:
        if cell.cell_type == "markdown":
            word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
    print("Word count: " + str(word_count) + ". Limit is 2000 words.")

In [ ]:
# This should be the final output of your notebook.
# Edit filename to be the same as this filename and then run.
# Save your file before running this code.

this_file_name = "CW-IEFT-template.ipynb" # Enter name of this file here
printWordCount(this_file_name)

Word count: 295. Limit is 2000 words.
